# kmp算法

在练习 KMP 算法时，请注意 KMP 的核心思想与「前缀函数（next数组）」的构建和使用方式。

### 一、KMP 核心思想

KMP 的目标是在主串 `s` 中查找模式串 `p` 的第一次出现位置，时间复杂度为 O(n + m)，避免暴力匹配中每次失配都将主串回退的开销。

**核心优化点：当匹配失败时，主串指针不回退，模式串根据 next 数组跳转到合适位置继续匹配。**

---

### 二、KMP 主要结构

#### 1. 构造 next 数组

- `next[i]` 表示：`p[0:i]` 这个子串（不含 `i`）中，**最长相等前后缀**的长度。
- 若当前字符 `p[i] != p[j]`，则令 `j = next[j]`（此时已经经过j++和i++了），即让模式串跳回上一个前后缀匹配位置。
- 本质上是回退到**次最长**相等前后缀的位置。
- 通过「双指针」构造，j 表示前后缀长度，i 向右走构造 next 表。

> 特殊初始化：`next[0] = -1`  
> 表示第一个字符失配时，模式串跳转到 -1（即主串向右滑一位，模式串重头匹配）

### 2. 构造 nextval 数组
- `nextval[i]` 表示在`next[i]`的基础上,递归到 `nextval[j]`，即在 `next` 数组的基础上进一步优化。
- 若经过i++和j++后，`p[i] = p[j]`，则 `nextval[i] = nextval[j]`；否则 `nextval[i] = j`。这是从递归的角度来理解的： 
  - 假设主串和模式串都已经匹配到 `i` 和 `j`，如果 `s[i] != p[j]`，那么j就要回退到 `next[j]`，（此时没有经过i++和j++），但是如果`p[j] = p[next[j]]`，那么`s[i]!= p[next[j]]`仍然成立，所以我们可以跳过这个位置，而是递归地来到`nextval[j]`，即 `nextval[i] = nextval[j]`（替代了`next[i] =j`）
  - 也就是说如果next[j]不符合要求，就递归到next[next[j]]，甚至next[next[next[j]]]，直到找到一个符合要求的nextval[j]，或者j==0为止。
  - 本质上是回退到**次最长**相等前后缀的n次方位置，直到p[j]!=p[nextval[j]]。
- 需要注意的是，`nextval` 数组的构造可以有效减少不必要的比较次数。
- `nextval` 数组的构造过程与 `next` 数组类似，但在每次失配时，`j` 的值会根据 `nextval[j]` 进行更新，从而跳过一些不必要的比较。
- 手算 `nextval` 数组时，可以往前不断递归next[j]、next[next[j]]，直到T.ch[j] != T.ch[next[j]]为止。
- 例如：见《数据结构与算法分析：C语言》p109 2.应用题(1).
#### 3. 主匹配过程

- 两个指针 `i`、`j` 遍历主串 `s` 和模式串 `p`。
- 若匹配成功：`s[i] == p[j]`，则 `i++` 和 `j++`。
- 若失配：`j = next[j]`，即模式串跳回 next 指定位置。本质上是回退到**次最长**相等前后缀的位置。
- 如果next改用nextval数组，失配时 `j = nextval[j]`。本质上是回退到**次最长**相等前后缀的n次方位置，直到p[j]!=p[nextval[j]]。
- 直到 `j == len(p)`，说明成功匹配，返回 `i - j` 为匹配起始位置。

---

### 三、记忆口诀

> - 主串 i 不回退，模式串 j 借助 next 回退  
> - next 表构造时：`j = next[j]`，直到找到匹配或 `j == -1`  
> - 匹配成功条件是：`j == len(p)`，返回 `i - j`

---

### 四、复杂度分析

- 构造 next 数组：每个位置最多进出一次，时间复杂度 O(m)
- 匹配过程：主串每个字符最多比较一次，时间复杂度 O(n)

**总时间复杂度：O(m + n)**，比朴素匹配的 O(mn) 要高效得多。

---

### 五、和暴力法的差异对比

| 项目         | 暴力匹配      | KMP 匹配           |
|--------------|----------------|---------------------|
| 匹配失败时   | 主串 i 回退    | 主串 i 不回退       |
| 模式串跳转位置 | 重新从头匹配  | 通过 next[j] 跳转  |
| 时间复杂度   | O(mn)          | O(m + n)            |

---

### 六、常见错误点

1. 忘记初始化 `next[0] = -1`
2. 构建 next 数组时 i、j 顺序混乱
3. 匹配成功后忘记返回 `i - j`
4. 把 next 数组理解成匹配失败时主串应该跳到的地方（其实是模式串跳转位置）

---

### 七、适合 KMP 的题目特点

- 需要在一个长字符串中查找一个短字符串（子串）的位置
- 字符串匹配类题目，如 LeetCode：
  - [28. 找出字符串中第一个匹配项的下标](https://leetcode.cn/problems/find-the-index-of-the-first-occurrence-in-a-string/)
  - [1392. 最长快乐前缀](https://leetcode.cn/problems/longest-happy-prefix/)
  - [214. 最短回文串](https://leetcode.cn/problems/shortest-palindrome/)（构造型）

---

> 💡 建议：掌握 KMP 的过程不必一蹴而就，先理解「匹配失败时模式串如何回退」，再推导 next 数组的构造逻辑，最后配合实例反复调试观察 `i, j, next[j]` 的变化，会有豁然开朗之感。

## 正式代码

get_next

In [ ]:
def get_next(p):
    """
    构建模式串 p 的 next 数组，也就是前缀表（即i之前的字符串的最长相等前后缀长度）。
    用长短双指针 i 和 j,j在前，i在后，i负责往前走，j负责记录当前位置的最长相等前后缀长度。
    如果字符不相等，j 回退到 next[j]，直到找到一个相等的前缀后缀，或者 j 回退到 -1。
    next[i] 表示 p[0:i-1] 的最长相等前后缀长度。本质上是告诉我们,kmp算法中，当模式串的j指针与主串匹配失败时，模式串应该回退到哪个位置。
    - 使用经典定义 next[0] = -1
    - 当 p[i] != p[j] 时，利用 next[j] 继续回退 j
    """
    nxt = [0] * len(p) # 初始化 next 数组,长度为 len(p),初始值为 0
    nxt[0] = -1 # 让第一个字符的 next 值为 -1
    i, j = 0, -1 # i和j分别是快慢指针，i在前，j在后，i负责往前走。j负责记录当前位置i之前的最长相等前后缀长度。注意j要初始化为-1，因为第一个字符的 next 值为 -1。
    while i < len(p) - 1: # 只处理i 的[0,len(p)-1)], 因为未来会通过i+1来更新next[i+1]。
        if j == -1 or p[i] == p[j]:  # 当j = -1 或者  p[i] == p[j]时，i 和 j 同时往前走
            print(f"i:{i},j:{j}")
            i += 1
            j += 1
            nxt[i] = j # 更新 next[i] 为 j，表示 p[0:i-1] 的最长相等前后缀长度为 j
            print(f"i:{i},j:{j},next[{i}]={j}")
        else: # 否则。
            print('j: ',j)
            # 这一步是精髓，不回退i，而是回退j。本质上是回退到次最长相等前后缀的位置。
            j = nxt[j]  # j回退到 next[j]，相当于把j回退到上一个相等的前后缀的位置，重新尝试用更短的前后缀匹配。
            # 举例：如果 p = 'abadcababe', i = 8,j =3, 此时i和j前面都有'aba',但是p[i] != p[j]，所以j = nxt[j] = 1 ,
            # 相当于j回退到j前面的最长相等前后缀的下一个位置，指向'b'，而不用指向第0个位置从头开始。接下来继续比较p[i]和p[j]。
            # 更深层的理解：此时，j相当于来到了i之前的次最长相等前后缀的下一个位置。为什么能这么做呢？因为i的前面就是'aba',此时j的前面也是'aba'，作为最长相等前后缀。
            # 那么如果这个最长相等前后缀中，存在着长度＞0的最长相等前后缀（最长相等前后缀的嵌套），那么就可以作为i之前的次最长相等前后缀。
            # 在这个例子中，就让j退到次最长相等前后缀的下一个位置也就是'b'，继续比较p[i]和p[j]。此时j的前面刚好是'a',i的前面也是'a'，这符合取次最长相等前后缀的结果。
            print('next[j]: ',j)
    print('--')
    return nxt
pattern = "abadcababe"
print(get_next(pattern))
# 输出：[-1, 0, 0, 1, 0, 0, 1, 2, 3, 2]

i:0,j:-1
i:1,j:0,next[1]=0
i:1,j:0
i:2,j:1,next[2]=1
--
[-1, 0, 1]


get_nextval

In [ ]:
def get_nextval(p):
    """
    构建模式串 p 的 nextval 数组，nextval 数组的定义和 next 数组类似，优化了 next 数组的重复值。
    本质上是递归：如果 +1 后的字符仍然相等，则 nextval[i] = nextval[j]（而不是j)，避免了重复值。
    """
    nxt = [0] * len(p) # 初始化 next 数组,长度为 len(p),初始值为 0
    nxt[0] = -1 # 让第一个字符的 next 值为 -1
    i, j = 0, -1 # i和j分别是快慢指针，i在前，j在后，i负责往前走。j负责记录当前位置i之前的最长相等前后缀长度。注意j要初始化为-1，因为第一个字符的 next 值为 -1。
    while i < len(p) - 1: # 只处理i 的[0,len(p)-1)], 因为未来会通过i+1来更新next[i+1]。
        if j == -1 or p[i] == p[j]:  # 当j = -1 或者  p[i] == p[j]时，i 和 j 同时往前走
            print(f"i:{i},j:{j}")
            i += 1
            j += 1
            if p[i] != p[j]:  # 如果+1后的字符不相等
                nxt[i] = j # 更新 next[i] 为 j，表示 p[0:i-1] 的最长相等前后缀长度为 j
            else: # !!!! 这里就是与get_next函数的区别。其他地方是一样的。
                nxt[i] = nxt[j] ## 如果+1后的字符仍然相等，则 nextval[i] = nextval[j]，表示 p[0:i-1] 的最长相等前后缀长度为 j
            print(f"i:{i},j:{j},next[{i}]={nxt[i]}")
        else: # 否则。
            print('j: ',j)
            # 这一步是精髓，不回退i，而是回退j。本质上是回退到次最长相等前后缀的位置。
            j = nxt[j]
            # j回退到 next[j]，相当于把j回退到上一个相等的前缀的位置，重新尝试用更短的前缀匹配。
            print('next[j]: ',j)
    print('--')
    return nxt
pattern = "abadcababe"
print(get_nextval(pattern))

i:0,j:-1
i:1,j:0,next[1]=0
j:  0
next[j]:  -1
i:1,j:-1
i:2,j:0,next[2]=-1
i:2,j:0
i:3,j:1,next[3]=1
j:  1
next[j]:  0
j:  0
next[j]:  -1
i:3,j:-1
i:4,j:0,next[4]=0
j:  0
next[j]:  -1
i:4,j:-1
i:5,j:0,next[5]=-1
i:5,j:0
i:6,j:1,next[6]=0
i:6,j:1
i:7,j:2,next[7]=-1
i:7,j:2
i:8,j:3,next[8]=3
j:  3
next[j]:  1
i:8,j:1
i:9,j:2,next[9]=2
--
[-1, 0, -1, 1, 0, -1, 0, -1, 3, 2]


kmp

In [ ]:
def kmp(s, p):
    ''' 
    kmp算法.返回s中第一次出现p的位置,如果没有出现返回-1。
    和get_next函数很相似，i负责往前走，j负责记录当前位置的最长相等前后缀长度，相当于长短双指针，而且i不回退，j回退到next[j]。
    不同之处在于：
        1.while循环条件
        2. j初始化为0而不是-1
        3. 不需要对nxt赋值
        4. 需要额外在最后判断j是否走到了p的末尾，并返回i-j。
    KMP 算法的核心思想：
         -用 next 数组保存「当模式串匹配失败时，下一个可以尝试匹配的位置」，避免暴力回退。
        - get_next 的核心思想是：
            - 如果 p[i] == p[j]，那么 next[i+1] = j+1；
            - 如果不等，通过 j = next[j] 不断寻找次级前后缀；
            - next[0] = -1 意味着 j 退无可退，需要重新开始。
        - 主匹配阶段（kmp 函数）中，i 只前进不回退，j 回退到 next[j]，时间复杂度 O(m+n)。
    '''
    nxt = get_next(p) # 获取模式串 p 的 next 数组
    # nxt = get_nextval(p) # 获取模式串 p 的 next 数组
    # 以上两种方法都可以，get_nextval方法更快，但是get_next方法更容易理解。

    i , j = 0,0 # i 和 j 分别指向文本串 s 和模式串 p 的当前匹配位置, 都初始化为0。
    while i < len(s) and j < len(p): # 只要 i 和 j 都没有越界就继续循环
        if j == -1 or s[i] == p[j]: # 如果 j == -1 或者 s[i] == p[j]，i 和 j 同时往前走
            i += 1
            j += 1
        else: # 否则，j 回退到 next[j]
            print('j: ',j)
            j = nxt[j]
            print('next[j]: ',j)
        print(i,j) 
    if j == len(p): # 如果 j 走到了 p 的末尾，说明匹配成功。
        return i - j # 返回匹配成功的位置。此时i走到了匹配成功的位置的下一个位置，j也走到了p的末尾也就是匹配成功的位置的下一个位置，
        # 等价于 return i - len(p)
        # 所以 s中第一次出现p的字符串末尾是i-1, 又因为此时j==len(p),所以  s中第一次出现p的字符串末尾位置 减去 p的长度-1 = i-1-(len(p)-1) = i-len(p) = i-j，就是s中第一次出现p的字符串开始位置。
    return -1   # 否则，返回 -1
s = 'caccacccaccacb'
p =       'caccacb'
print('kmp result:', kmp(s,p))
# 输出：7

i:0,j:-1
i:1,j:0,next[1]=0
j:  0
next[j]:  -1
i:1,j:-1
i:2,j:0,next[2]=0
i:2,j:0
i:3,j:1,next[3]=1
j:  1
next[j]:  0
i:3,j:0
i:4,j:1,next[4]=1
i:4,j:1
i:5,j:2,next[5]=2
i:5,j:2
i:6,j:3,next[6]=3
--
1 1
2 2
3 3
4 4
5 5
6 6
j:  6
next[j]:  3
6 3
7 4
j:  4
next[j]:  1
7 1
j:  1
next[j]:  0
7 0
8 1
9 2
10 3
11 4
12 5
13 6
14 7
kmp result: 7
